In [1]:
%load_ext autoreload
%autoreload
import os
from tqdm import tqdm
import numpy as np
import torch
import pandas as pd
from sklearn.preprocessing import normalize
from RS.utils.dataset import Dataset as Mydataset
from RS.utils.dataset import combine_multi_domain
from RS.utils.dataset import user_item_clustering
from RS.utils.mf import ALS_MF
from RS.utils.dictutils import *
dataroot = os.path.join("data")
d = torch.device('cuda:4')

In [2]:
datafolder = {
    "training_user_course":os.path.join(
        dataroot,"course","train.csv"
    ),
    "training_user_book":os.path.join(
        dataroot,"book","user_cate3_train.csv"
    ),
    "testing_user_course":os.path.join(
        dataroot, "course", "test.csv"
    ),
    "testing_user_book":os.path.join(
        dataroot, "book", "user_cate3_test.csv"
    )
}


dataset = Mydataset(datafolder=datafolder)

build dataset
read training_user_course:data/course/train.csv
..OK
read training_user_book:data/book/user_cate3_train.csv
..OK
read testing_user_course:data/course/test.csv
..OK
read testing_user_book:data/book/user_cate3_test.csv
..OK


## Clustering

###  book

In [4]:
user_book_df = dataset.getdata(
    "training_user_book", normalize_value=True
)
user_book_df.head()

,uid,0,1,2,3,4,5,6,7,8,...,990,991,992,993,994,995,996,997,998,999
0,400110002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,400110003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,400110005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,400110006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,400110007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### user - book

In [21]:
user_item_clustering(
    User_Item_df=user_book_df,
    num_clusters=100,
    savingpath=os.path.join(
        "result","CBMF","clustering","bookdataset"
    ),
    d=d
)

random loss: 102.3496293036183


 60%|██████    | 3/5 [00:04<00:02,  1.35s/it, currentbest=0.000, mse=0.000, improve=0.0000, early=3]  


user
(20575, 40)
clutsering ..
OK ..
visualization ..


/home/tch/.conda/envs/tenv/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:795: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/tch/.conda/envs/tenv/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:805: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


item
(1000, 40)
clutsering ..
OK ..
visualization ..


/home/tch/.conda/envs/tenv/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:795: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/tch/.conda/envs/tenv/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:805: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


#### books clustering by first 2 layer 

In [6]:
cate = user_book_df.drop(columns=['uid']).columns.tolist()
step = 10
layer2_cate = {
    k:cate[v:v+step] 
    for k, v in enumerate(range(0, len(cate), step))
}
writejson(
    layer2_cate, 
    jsfilepath=os.path.join(
        "result","CBMF","clustering","bookdataset","itemclustering",
        "layer2.json"
    )
)


### user - course 

In [16]:
user_course_df = combine_multi_domain(
    Dataset=dataset, 
    domains=[
        [
            ("training_user_course",False,False),
            ("testing_user_course",False,False)
        ]
    ]
)
user_course_df.head()

name : training_user_course, mask: False, normalization: False
    normalization along row : False
name : testing_user_course, mask: False, normalization: False
    normalization along row : False


,uid,1011101,1011111,1011112,1011113,1011114,1011115,1011116,1011117,1011118,...,7507019,7507020,7507021,7507022,7507023,7507024,7507025,7507026,7507027,7507028
0,400110002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,400110003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,400110005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,400110006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,400110007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
user_item_clustering(
    User_Item_df=user_course_df,
    num_clusters=100,
    savingpath=os.path.join(
        "result","CBMF","clustering","coursedataset"
    ),
    d=d
)

random loss: 102.23353621936171


 80%|████████  | 4/5 [00:40<00:10, 10.03s/it, currentbest=0.003, mse=0.003, improve=0.0000, early=3]  


user
(21829, 40)
clutsering ..
OK ..
visualization ..


/home/tch/.conda/envs/tenv/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:795: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/tch/.conda/envs/tenv/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:805: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


item
(7054, 40)
clutsering ..
OK ..
visualization ..


/home/tch/.conda/envs/tenv/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:795: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/tch/.conda/envs/tenv/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:805: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


## Cross domain cluster level matrix

In [8]:
book_cluster = loadjson(
    os.path.join(
        "result","CBMF",
        "clustering","bookdataset",
        "itemclustering", "layer2.json"
    )
)

course_cluster = loadjson(
    os.path.join(
        "result","CBMF",
        "clustering","coursedataset",
        "itemclustering", "cluster.json"
    )
)


user_course_cluster = loadjson(
    os.path.join(
        "result","CBMF",
        "clustering","coursedataset",
        "userclustering", "cluster.json"
    )
)

user_book_cluster = loadjson(
    os.path.join(
        "result","CBMF",
        "clustering","bookdataset",
        "userclustering", "cluster.json"
    )
)

user_book_cluster[
    len(user_book_cluster.keys())
] = dataset.getdata("testing_user_book").uid.tolist()

In [9]:
%autoreload
from RS.utils.dataset import cluster_level_matrix

### Book 

In [10]:
courseUser_book = cluster_level_matrix(
    R=dataset.getdata("training_user_book"),
    user_cluster=user_course_cluster,
    item_cluster=book_cluster
)
print(courseUser_book.shape)

(100, 100)


100%|██████████| 100/100 [00:05<00:00, 19.54it/s]

5677/10000
(100, 100)


In [11]:
bookUser_book = cluster_level_matrix(
    R=dataset.getdata("training_user_book"),
    user_cluster=user_book_cluster,
    item_cluster=book_cluster
)

(101, 100)


100%|██████████| 101/101 [00:05<00:00, 19.37it/s]

7116/10100


In [12]:
user_book_cluster_level = np.concatenate(
    [courseUser_book, bookUser_book ],
    axis=0
)
user_book_cluster_level = normalize(
    user_book_cluster_level, norm="l1",axis=1
)
print(user_book_cluster_level.shape)

(201, 100)


In [13]:
np.save(
    os.path.join('result',"CBMF","clustering","L2bookC"),
    user_book_cluster_level
)

### Course

In [17]:
courseUser_course = cluster_level_matrix(
    R=user_course_df,
    user_cluster=user_course_cluster,
    item_cluster=course_cluster
)

(100, 100)


100%|██████████| 100/100 [00:06<00:00, 14.97it/s]

5229/10000


In [18]:
bookUser_course = cluster_level_matrix(
    R=user_course_df,
    user_cluster=user_book_cluster,
    item_cluster=course_cluster
)

(101, 100)


100%|██████████| 101/101 [00:07<00:00, 13.81it/s]

3335/10100


In [19]:
user_course_cluster_level = np.concatenate(
    [courseUser_course, bookUser_course ],
    axis=0
)
print(user_course_cluster_level.shape)

(201, 100)


In [20]:
np.save(
    os.path.join('result',"CBMF","clustering","courseC"),
    user_course_cluster_level
)

### Combine

In [21]:
cb = np.concatenate(
    [user_book_cluster_level,user_course_cluster_level],
    axis=1
)
print(cb.shape)

(201, 200)


In [71]:
np.save(
    os.path.join('result',"CBMF","clustering","l2_cb"),
    cb
)